출처: https://github.com/yoonkt200/python-data-analysis/blob/master/chapter2/(4%EC%87%84%20%ED%81%AC%EB%A1%A4%EB%A7%81%20%EB%B3%80%EA%B2%BD%20%EC%98%88%EC%A0%9C)%20naver-news-crawling.ipynb

In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re

# 크롤링할 사이트 주소를 입력합니다.
source_url = "https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100"

# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
driver = webdriver.Chrome()  # for Windows -> 셀레니움 update, path필요X
driver.get(source_url)
req = driver.page_source
soup = BeautifulSoup(req, "html.parser")
cluster_text = soup.find_all(name="div", attrs={"class":"cluster_text"})

In [3]:
cluster_text # "cluster_text" class속성을 가진 요소들과 그 자식요소들이 묶여서 원소 1개임
len(cluster_text) # 4

4

In [4]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_urls = []
for index in range(0, len(cluster_text)):
    cluster = cluster_text[index]
    news_url = cluster.find(name="a", attrs={"class":"cluster_text_headline nclicks(pol.sera)"}) # class name 예제랑 조금 바뀜
    if news_url is not None:
        page_urls.append(news_url.get("href"))

# 중복 url을 제거합니다.
page_urls = list(set(page_urls))

# 다섯 개의 페이지를 출력합니다.
for page in page_urls[:5]:
    print(page)

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

https://n.news.naver.com/mnews/hotissue/article/449/0000251687?type=series&cid=1088492
https://n.news.naver.com/mnews/hotissue/article/449/0000251547?type=series&cid=1088492
https://n.news.naver.com/mnews/hotissue/article/449/0000251827?type=series&cid=1088492
https://n.news.naver.com/mnews/hotissue/article/449/0000251912?type=series&cid=1088492


In [6]:
# 페이지 내 텍스트 구조 확인
driver = webdriver.Chrome()
driver.get(page_urls[0])
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')
title_area = soup.find(name="div", attrs={"class":"media_end_head_title"})
title = title_area.find_all('h2')[0]
content_paragraphs = soup.find(name="div", attrs={"class":"newsct_article"})
content_corpus = content_paragraphs.text

print(title.text)
print("\n")
print(content_corpus)

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

[아는기자]교육부 ‘이권 카르텔’ 지적…수능 쉬워지나




			[앵커]아는 기자, 아자 정치부 조아라 기자 나왔습니다.  Q. 교육이라는 게 워낙 민감한 이슈인데요. 툭 터져나온 감이 있어요?  윤석열 대통령, 어제 이주호 교육부 장관으로부터 교육 개혁 관련 보고를 받았는데요.  이 자리에서 윤 대통령이 사교육비 부담을 지적하며, 수능을 언급했습니다.  [이주호 / 사회부총리 겸 교육부 장관(어제)] "(윤 대통령은) 대학수학능력시험과 관련하여 변별력은 갖추되 학교 수업만 열심히 따라가면 문제를 풀 수 있도록 출제하고 학교 수업에서 다루지 않은 내용은 출제에서 배제하라고 말씀하였습니다."  이 이야기를 들으면 여러분은 어떤 느낌이 드십니까?  많은 이들이 쉽게 내겠구나라고 생각했고, 이른바 '물수능'이 될 거라는 전망이 쏟아졌습니다.  대통령실은 4시간 만에 정정 공지를 냈는데요.  그러면서 이주호 장관에게 한 대통령의 추가 발언을 소개했습니다.  "공교육 교과과정에서 다루지 않는 부분의 문제를 수능에서 출제하면, 사교육에 의존하라는 것 아니냐" "교육당국과 사교육 산업이 한 편, 즉 카르텔이란 말인가"라고 지적했다고요. 윤 대통령 지시는 수능 난이도에 대한 게 아니라 입시에서의 공정성 복원에 초점이 맞춰져 있다는 게 대통령실의 설명입니다.  Q. "이권 카르텔"이라고 했던 데 정확히 뭘 의심하는 건가요?  오늘 교육부 차관의 이 발언을 들으면 이해가 되실텐데요.  이주호 장관은 '교육 당국과 사교육산업이 한통속으로 생각된다'는 대통령의 지적을 매우 무겁게 받아들이고 책임감을 느끼고 있다 그러니까 교육당국은 학교에서 알려주지 않는 문제를 내고, 사교육계는 족집게 수능 기술을 알려주면서 돈을 버는 유착관계가 있다고 의심하는 겁니다.  윤 대통령은 검사 시절부터 이런 유착 관계에 대해 강한 문제 의식을 갖고 있었던 것으로 전해졌습니다.  Q. 시청자들은 이런 게 궁금하신거 같아요. 카르텔이라고 볼만한 근거가 있나요?  아직 뚜렷한 물증까지 제시하진 못하는데요.

**네이버 뉴스 헤드라인 크롤링**

In [7]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비합니다.
columns = ["title", "content_text"]
df = pd.DataFrame(columns=columns)

# 각 페이지별 '제목', '본문' 정보를 데이터 프레임으로 만듭니다.
for page_url in page_urls:

    # 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
    driver = webdriver.Chrome()  # for Windows
    driver.get(page_url)
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser")

    title_area = soup.find(name="div", attrs={"class": "media_end_head_title"})
    title = title_area.find_all('h2')[0]
    content_paragraphs = soup.find(name="div", attrs={"class": "newsct_article"})
    content_corpus = content_paragraphs.text

    # 제목 정보에서 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""

    # 모든 정보를 하나의 데이터 프레임에 저장합니다.
    row = [row_title, content_corpus]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

    # 크롤링에 사용한 브라우저를 종료합니다.
    driver.close()

In [8]:
df.head()

,title,content_text
0,[아는기자]교육부 ‘이권 카르텔’ 지적…수능 쉬워지나,"\n\n\t\t\t[앵커]아는 기자, 아자 정치부 조아라 기자 나왔습니다. Q. ..."
1,"[아는기자]尹 대통령, ‘태양광 장사’에 직접 나선 이유는?","\n\n\t\t\t[앵커]아는 기자, 아자 정치부 조영민기자 나왔습니다. Q1. ..."
2,[아는기자]한국 vs 사우디 2파전…표차는?,"\n\n\t\t\t[앵커]아는기자, 아자 대통령실 출입하는 조아라 기자 나왔습니다...."
3,[아는기자]“30분 숨 쉴 틈 없이”…엑스포 유치 4차 PT 전략은?,"\n\n\t\t\t[앵커]아는기자, 아자 대통령실 출입하는 조아라 기자 나왔습니다...."
